# 实现路线思考：
1. 集合所有短语，让模型进行自动识别分类
2. 根据向量进行自动化分类

# 一、数据准备

## 1. 从mongo数据库中获取数据，产生对应json数据格式

In [22]:
import pymongo
import pandas as pd
# MongoDB连接配置
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
kinyo_db = mongo_client["kinyo_db"]
kinyo_reviews_collection = kinyo_db["kinyo_new_reviews"]
kinyo_llm_results_collection = kinyo_db["kinyo_llm_results"]
kinyo_data_result = kinyo_db["kinyo_data_result"]

import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体为黑体
plt.rcParams['axes.unicode_minus'] = False    # 正常显示负号

In [23]:
# 1. 用户画像统计
# 获取已处理的评论ID集合
# 设置当前实验的 test_version 和 solution
current_test_version = "kinyo-data-10"
current_solution = "AI自打标：不限定"
print("test_version:", current_test_version)
print("solution:", current_solution)
kinyo_llm_results_collection.find_one({"test_version": current_test_version, "solution": current_solution})

test_version: kinyo-data-10
solution: AI自打标：不限定


{'_id': ObjectId('6809c91ef187ea1414057e29'),
 'comment': '- 声音很大，外观也好看',
 'product_topic_result': [{'topic': '音质',
   'polarity': '好评',
   'confidence': 0.9,
   'related_text': '声音很大'},
  {'topic': '外观设计',
   'polarity': '好评',
   'confidence': 0.8,
   'related_text': '外观也好看'}],
 'keyphrases': ['声音很大', '外观也好看'],
 'user_profile': {},
 'token_usage': 1132,
 'review_id': 'e136dabe-e0a8-41d1-9e7e-f9381abd6100',
 'analysis_time': '2025-04-24T13:16:14.823949',
 'test_version': 'kinyo-data-10',
 'solution': 'AI自打标：不限定',
 'model': 'gpt-3.5-turbo'}

## 2. 矫正分类模型

In [24]:
# imports
import ast  # for converting embeddings saved as strings back to arrays
from openai import OpenAI # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
import os # for getting API token from env variable OPENAI_API_KEY
from scipy import spatial  # for calculating vector similarities for search
import json
# create a list of models 
GPT_MODELS = ["gpt-4o", "gpt-4o-mini"]
# models
EMBEDDING_MODEL = "BAAI/bge-base-zh"

In [25]:
# Set the proxy URL and port
proxy_url = 'http://127.0.0.1'
proxy_port = 6465 # !!!please replace it with your own port

# Set the http_proxy and https_proxy environment variables
os.environ['http_proxy'] = f'{proxy_url}:{proxy_port}'
os.environ['https_proxy'] = f'{proxy_url}:{proxy_port}'

client = OpenAI(api_key=os.getenv('OPEN_AI_KEY'))

In [26]:
def prepare_correct_category_prompt(origin_category: str,category_type: str):
    prompt = f"""# 任务
请你根据下列商品的原始分类信息，结合语义含义，归纳总结出一组【简明、具体、互不重复且有明显区别】的典型分类。要求如下：

- 分类名称要用普通人日常生活中常说的词语，而不是笼统或抽象的表达。
- 分类之间要有清晰的区分，不要出现含糊或重复的类别。
- 尽量避免使用行业术语或官方用语，确保每个分类都让大众一看就明白。
- 如果原始分类有相似但细微差别的，可以合并为一个大家都熟悉的类别；如果差异明显，则分别列出。
- 请确保所有原始分类都能被覆盖，不遗漏、不冗余。

# 原始分类信息：
{origin_category}
# 分类场景：
{category_type}
# 输出格式
请用如下JSON格式输出：
{{
  "categories": ["分类1", "分类2", ...]
}}
"""
    return prompt


In [27]:
from typing import Dict, Any  # 添加类型注解导入
# 调用模型获取结果
def get_correct_category(origin_category:str,category_type:str,model:str="gpt-4o-mini"):

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "你是一个专业的分类专家"},
                {"role": "user", "content": prepare_correct_category_prompt(origin_category=origin_category,category_type=category_type)}
            ],
            stream=False,
            response_format={'type': 'json_object'}
        )

        # 获取 token 消耗数量
        token_usage = response.usage.total_tokens if hasattr(response, 'usage') else None
        # 解析内容
        content = response.choices[0].message.content
        parsed_content = json.loads(content)  # 解析内容
        parsed_content['token_usage'] = token_usage
        return parsed_content
    except Exception as e:
        print(f"Error: {e}")
        return None

# 二、用户画像矫正

## 0. 用户画像数据统计

In [28]:
from collections import Counter
import json

def profile_stats(collection, test_version, solution, fields):
    cursor = collection.find({
        "test_version": test_version,
        "solution": solution
    })

    # 初始化每个字段的计数器
    field_counters = {field: Counter() for field in fields}
    total = 0

    for doc in cursor:
        user_profile = doc.get("user_profile", {})
        for field in fields:
            value = user_profile.get(field, "")
            field_counters[field][value] += 1
        total += 1

    # 组织结果
    result = {}
    for field, counter in field_counters.items():
        field_result = []
        for value, count in counter.items():
            percent = round(count / total, 4) if total > 0 else 0
            field_result.append({
                field: value,
                "count": count,
                "percent": percent
            })
        result[field] = field_result

    print(json.dumps(result, ensure_ascii=False, indent=2))
    return result

# 用法示例
fields = [
    "gender", "occupation", "consumption_scene", "consumption_motivation",
    "consumption_frequency", "consumption_satisfaction", "consumption_unsatisfaction"
]
profile_stats_json = profile_stats(kinyo_llm_results_collection, current_test_version, current_solution, fields)

{
  "gender": [
    {
      "gender": "",
      "count": 8,
      "percent": 0.8
    },
    {
      "gender": "女",
      "count": 2,
      "percent": 0.2
    }
  ],
  "occupation": [
    {
      "occupation": "",
      "count": 10,
      "percent": 1.0
    }
  ],
  "consumption_scene": [
    {
      "consumption_scene": "",
      "count": 8,
      "percent": 0.8
    },
    {
      "consumption_scene": "室内",
      "count": 2,
      "percent": 0.2
    }
  ],
  "consumption_motivation": [
    {
      "consumption_motivation": "",
      "count": 8,
      "percent": 0.8
    },
    {
      "consumption_motivation": "满意的购物体验",
      "count": 1,
      "percent": 0.1
    },
    {
      "consumption_motivation": "购物体验满意",
      "count": 1,
      "percent": 0.1
    }
  ],
  "consumption_frequency": [
    {
      "consumption_frequency": "",
      "count": 8,
      "percent": 0.8
    },
    {
      "consumption_frequency": "首次",
      "count": 2,
      "percent": 0.2
    }
  ],
  "consumption_sati

## 1. 用户画像数据矫正

In [32]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np


In [33]:
def normalize_user_profile_field(
    field_name,                     # 字段名称，如 "occupation"、"gender" 等
    category_type,                  # 分类类型，如 "职业"、"性别" 等
    profile_stats_json,             # 统计数据
    collection,                     # MongoDB集合
    test_version,                   # 测试版本
    solution,                       # 解决方案
    model_name="gpt-4o-mini",       # 使用的模型
    embedding_model=EMBEDDING_MODEL # 嵌入模型
):
    """
    对用户画像字段进行归一化处理并更新到MongoDB
    
    参数:
    field_name: 需要处理的字段名称，如 "occupation"
    category_type: 分类类型，如 "职业"
    profile_stats_json: 包含统计信息的JSON
    collection: MongoDB集合
    test_version: 测试版本
    solution: 解决方案
    model_name: 使用的LLM模型
    embedding_model: 使用的嵌入模型
    
    返回:
    DataFrame: 包含原始类别和归属类别的数据框
    """
    # 1. 获取原始数据列表
    original_list = [
        item[field_name]
        for item in profile_stats_json[field_name]
    ]
    
    # 2. 获取标准分类
    category_names = get_correct_category(
        origin_category=",".join(original_list),
        category_type=category_type,
        model=model_name
    )["categories"]
    print(f"重新的{category_type}分类：", category_names)
    
    # 3. 向量化
    model = SentenceTransformer(embedding_model)
    category_vecs = model.encode(category_names)
    original_vecs = model.encode(original_list)
    
    # 4. 相似度归类
    results = []
    for orig, orig_vec in zip(original_list, original_vecs):
        if orig.strip() != '':
            # 非空值，计算相似度
            sims = [cosine_sim(orig_vec, cat_vec) for cat_vec in category_vecs]
            idx = np.argmax(sims)
            assigned_category = category_names[idx]
            results.append({"原始类别": orig, "归属类别": assigned_category})
            
            # 更新MongoDB
            new_field_name = f"user_profile.new_{field_name}"
            collection.update_many(
                {
                    "test_version": test_version,
                    "solution": solution,
                    f"user_profile.{field_name}": orig
                },
                {"$set": {new_field_name: assigned_category}}
            )
        else:
            # 空值处理为"未知"
            new_field_name = f"user_profile.new_{field_name}"
            collection.update_many(
                {
                    "test_version": test_version,
                    "solution": solution,
                    f"user_profile.{field_name}": orig
                },
                {"$set": {new_field_name: "未知"}}
            )
    
    # 5. 生成DataFrame
    df = pd.DataFrame(results)
    return df

# 余弦相似度计算函数
def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [34]:
# 处理职业字段
occupation_df = normalize_user_profile_field(
    field_name="occupation",
    category_type="职业",
    profile_stats_json=profile_stats_json,
    collection=kinyo_llm_results_collection,
    test_version=current_test_version,
    solution=current_solution
)
print("职业归类结果:")
print(occupation_df)

# 处理性别字段
gender_df = normalize_user_profile_field(
    field_name="gender",
    category_type="性别",
    profile_stats_json=profile_stats_json,
    collection=kinyo_llm_results_collection,
    test_version=current_test_version,
    solution=current_solution
)
print("性别归类结果:")
print(gender_df)

# 处理消费场景字段
consumption_scene_df = normalize_user_profile_field(
    field_name="consumption_scene",
    category_type="用户消费场景",
    profile_stats_json=profile_stats_json,
    collection=kinyo_llm_results_collection,
    test_version=current_test_version,
    solution=current_solution
)
print("消费场景归类结果:")
print(consumption_scene_df)

# 处理消费动机字段
consumption_motivation_df = normalize_user_profile_field(
    field_name="consumption_motivation",
    category_type="用户消费动机",
    profile_stats_json=profile_stats_json,
    collection=kinyo_llm_results_collection,
    test_version=current_test_version,
    solution=current_solution
)
print("消费动机归类结果:")
print(consumption_motivation_df)

# 处理消费频率字段
consumption_frequency_df = normalize_user_profile_field(
    field_name="consumption_frequency",
    category_type="用户消费频率",
    profile_stats_json=profile_stats_json,
    collection=kinyo_llm_results_collection,
    test_version=current_test_version,
    solution=current_solution
)
print("消费频率归类结果:")
print(consumption_frequency_df)

# 处理消费满意点字段
consumption_satisfaction_df = normalize_user_profile_field(
    field_name="consumption_satisfaction",
    category_type="用户消费满意点",
    profile_stats_json=profile_stats_json,
    collection=kinyo_llm_results_collection,
    test_version=current_test_version,
    solution=current_solution
)
print("消费满意点归类结果:")
print(consumption_satisfaction_df)

# 处理消费不满意字段
consumption_unsatisfaction_df = normalize_user_profile_field(
    field_name="consumption_unsatisfaction",
    category_type="用户消费不满意点",
    profile_stats_json=profile_stats_json,
    collection=kinyo_llm_results_collection,
    test_version=current_test_version,
    solution=current_solution
)
print("消费不满意点归类结果:")
print(consumption_unsatisfaction_df)


重新的职业分类： ['医疗', '教育', '工程', '科技', '艺术', '商业', '服务', '体育', '农业']
职业归类结果:
Empty DataFrame
Columns: []
Index: []
重新的性别分类： ['女性']
性别归类结果:
  原始类别 归属类别
0    女   女性
重新的用户消费场景分类： ['室内用品', '家居设备', '家庭装饰', '室内植物', '日常杂货']
消费场景归类结果:
  原始类别  归属类别
0   室内  室内用品
重新的用户消费动机分类： ['购物满意度']
消费动机归类结果:
      原始类别   归属类别
0  满意的购物体验  购物满意度
1   购物体验满意  购物满意度
重新的用户消费频率分类： ['偶尔购买', '经常购买']
消费频率归类结果:
  原始类别  归属类别
0   首次  偶尔购买
重新的用户消费满意点分类： ['无线功能', '充电便捷性', '音质优良']
消费满意点归类结果:
               原始类别  归属类别
0  UHF功能、充电线设计、音质表现  无线功能
1             UHF功能  无线功能
重新的用户消费不满意点分类： ['配送问题', '产品功能与价格不匹配']
消费不满意点归类结果:
       原始类别        归属类别
0    珍珠链未配送        配送问题
1  领夹麦功能与价格  产品功能与价格不匹配


# 三、标签数据矫正

In [35]:
def prepare_correct_category_prompt(origin_category: str, category_type: str):
    prompt = f"""# 任务
请你作为电商领域专家，根据下列商品的原始话题分类信息，结合语义含义，归纳总结出一组【具体、精准、层次分明】的商品话题分类体系。要求如下：

- 分类体系应保留电商评价中的重要细节和维度，如商品质量、外观、功能、性价比、物流、服务等。
- 分类名称要简洁明了，既要专业又要通俗易懂，便于普通消费者理解。
- 分类之间要有明确的界限，避免概念重叠，确保每个话题有其独特的关注点。
- 对于高频出现的话题（如物流、质量、价格等），可以进一步细分为更具体的子类别。
- 如果出现子类别，可以使用"/"分隔，例如："物流配送/速度"。
- 对于低频但有特色的话题，可以适当保留其独特性，不要过度合并。
- 请确保分类体系能覆盖所有原始话题，并且具有实用性和可扩展性。

# 原始话题分类信息：
{origin_category}

# 分类场景：
电商商品{category_type}评价分析

# 输出格式
请用如下JSON格式输出：
{{
  "categories": ["分类1", "分类2", ...]，所有的分类全部在列表中
}}
"""
    return prompt

In [36]:
from typing import Dict, Any  # 添加类型注解导入
# 调用模型获取结果
def get_correct_label_category(origin_category:str,category_type:str,model:str="gpt-4o-mini"):

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "你是一个专业的分类专家"},
                {"role": "user", "content": prepare_correct_category_prompt(origin_category=origin_category,category_type=category_type)}
            ],
            stream=False,
            response_format={'type': 'json_object'}
        )

        # 获取 token 消耗数量
        token_usage = response.usage.total_tokens if hasattr(response, 'usage') else None
        # 解析内容
        content = response.choices[0].message.content
        parsed_content = json.loads(content)  # 解析内容
        parsed_content['token_usage'] = token_usage
        return parsed_content
    except Exception as e:
        print(f"Error: {e}")
        return None

In [37]:
def normalize_product_topics(
    profile_stats_json,             # 统计数据
    collection,                     # MongoDB集合
    test_version,                   # 测试版本
    solution,                       # 解决方案
    model_name="gpt-4o-mini",       # 使用的模型
    embedding_model=EMBEDDING_MODEL # 嵌入模型
):
    """
    对product_topic_result中的topic字段进行归一化处理并更新到MongoDB
    
    参数:
    profile_stats_json: 包含统计信息的JSON
    collection: MongoDB集合
    test_version: 测试版本
    solution: 解决方案
    model_name: 使用的LLM模型
    embedding_model: 使用的嵌入模型
    
    返回:
    DataFrame: 包含原始类别和归属类别的数据框
    """
    # 1. 收集所有不同的topic
    all_topics = set()
    
    # 从MongoDB中获取所有不同的topic
    pipeline = [
        {"$match": {"test_version": test_version, "solution": solution}},
        {"$unwind": "$product_topic_result"},
        {"$group": {"_id": "$product_topic_result.topic"}}
    ]
    
    topic_results = list(collection.aggregate(pipeline))
    original_topics = [doc["_id"] for doc in topic_results if doc["_id"].strip() != '']
    
    # 2. 获取标准分类
    category_names = get_correct_label_category(
        origin_category=",".join(original_topics),
        category_type="扩音器",
        model=model_name
    )["categories"]
    print(f"重新的商品话题分类：", category_names)
    
    # 3. 向量化
    model = SentenceTransformer(embedding_model)
    category_vecs = model.encode(category_names)
    topic_vecs = model.encode(original_topics)
    
    # 4. 相似度归类
    results = []
    topic_mapping = {}  # 用于存储映射关系
    
    for topic, topic_vec in zip(original_topics, topic_vecs):
        if topic.strip() != '':
            # 非空值，计算相似度
            sims = [cosine_sim(topic_vec, cat_vec) for cat_vec in category_vecs]
            idx = np.argmax(sims)
            assigned_category = category_names[idx]
            results.append({"原始话题": topic, "归属类别": assigned_category})
            topic_mapping[topic] = assigned_category
    
    # 5. 更新MongoDB
    # 由于product_topic_result是数组，我们需要遍历每个文档并更新
    for doc in collection.find({"test_version": test_version, "solution": solution}):
        if "product_topic_result" in doc and isinstance(doc["product_topic_result"], list):
            updated_topics = []
            for topic_item in doc["product_topic_result"]:
                # 复制原始项
                updated_item = topic_item.copy()
                
                # 添加new_topic字段
                original_topic = topic_item.get("topic", "")
                if original_topic.strip() != '':
                    updated_item["new_topic"] = topic_mapping.get(original_topic, "未知")
                else:
                    updated_item["new_topic"] = "未知"
                
                updated_topics.append(updated_item)
            
            # 更新文档
            collection.update_one(
                {"_id": doc["_id"]},
                {"$set": {"product_topic_result": updated_topics}}
            )
    
    # 6. 生成DataFrame
    df = pd.DataFrame(results)
    return df

# 余弦相似度计算函数
def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [38]:
# 处理商品话题
topic_df = normalize_product_topics(
    profile_stats_json=profile_stats_json,
    collection=kinyo_llm_results_collection,
    test_version=current_test_version,
    solution=current_solution
)
print("商品话题归类结果:")
print(topic_df)

重新的商品话题分类： ['商品质量', '外观设计', '功能性能', '使用体验', '性价比', '价格', '物流服务', '客服服务']
商品话题归类结果:
        原始话题  归属类别
0        便携性  功能性能
1       续航表现  使用体验
2     无线耳麦性能  功能性能
3         价格    价格
4       客服态度  客服服务
5       外观设计  外观设计
6        性价比   性价比
7       功能多样  功能性能
8      UHF功能  功能性能
9       购物体验  使用体验
10        质量  商品质量
11      佩戴舒适  使用体验
12        音质  商品质量
13     有线麦外观  外观设计
14  领夹麦功能与价格  功能性能
15    购物平台比较   性价比
16      售后配送  物流服务
17     充电线设计  外观设计
18      物流服务  物流服务
19      品质感受  商品质量
20      品牌放心  商品质量
21      音质表现  使用体验
22      充电速度  使用体验


In [39]:
topic_df['归属类别'].value_counts()

归属类别
功能性能    5
使用体验    5
商品质量    4
外观设计    3
性价比     2
物流服务    2
价格      1
客服服务    1
Name: count, dtype: int64